In [4]:
#Use this to fix API calls and better parse xml 

In [7]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests
from urllib.parse import urlencode, quote
import xml.etree.ElementTree as ET
import xmltodict


In [1]:
def singlePointDataQuery(lat1, lon1, product, begin, end, Unit='m', optional_params=['wspd', 'wdir']):
	params = {
		'lat':lat1,
		'lon':lon1,
		'product':product
	}
	params2 = {'begin':begin,
	'end':end
	}
	params3 = {
		'Unit':Unit,
	}
	urlString = urlencode(params)
	subString = ''
	for key, value in params2.items():
		subString += '&'+str(key) + '=' + str(value)
	urlString+=subString
	for i in optional_params:
		params3[i] = i
	urlString +='&' + urlencode(params3)
	return urlString

In [2]:
def subGrid(centerPointLat, centerPointLon, distanceLat, distanceLon, resolutionSquare, product, begin, end, Unit='m', optional_params=['wspd', 'wdir']):
	#Split into 3 dictionaries, each are encoded in a different manner
	params = {
		'centerPointLat':centerPointLat,
		'centerPointLon':centerPointLon,
		'distanceLat':distanceLat,
		'distanceLon':distanceLon,
		'resolutionSquare':resolutionSquare,
		'product':product
	}
	#Begin/end has special encoding
	params2 = {'begin':begin,
	'end':end
	}
	params3 = {
		'Unit':Unit,
	}

	urlString = urlencode(params)
	subString = ''
	for key, value in params2.items():
		subString += '&'+str(key) + '=' + str(value)
	urlString+=subString
	for i in optional_params:
		params3[i] = i
	urlString +='&' + urlencode(params3)

	return urlString

In [3]:
def run_request(q):
	print(_url(q))
	resp = requests.get(_url(q))
	if resp.status_code != 200:
		# This means something went wrong.
		print(resp.status_code)
		raise ApiError('GET /tasks/ {}'.format(resp.status_code))
	return resp

In [4]:

#Main URL path
def _url(path=''):
    return 'http://www.weather.gov/forecasts/xml/sample_products/browser_interface/ndfdXMLclient.php?' + path


In [64]:
#Still in dev, now returns values for certain methods
def parseXml(data):
	outDict = {}
	#Load xml object into dictionary
	d = xmltodict.parse(data.content)	
	#use dataDict as base content dictionary, we only care about this
	dataDict = d['dwml']['data']
	print(dataDict)
#     print(dataDict.keys())
	#Put list of points  into outDict, lat and long are now called @latitude and @longitude
	for key in dataDict['location']:
		outDict[key['location-key']] = [{'loc':key['point']}]
	#Make a faux list called locationsList. This is iterated over to fill outDict for now because xml structure is annoying.
	locationsList = list(outDict.keys())
	for i in range(len(locationsList)):
		for j in range(len((dataDict['parameters'][i]['wind-speed']['value']))):
			outDict[locationsList[i]].append({'wind-speed':dataDict['parameters'][i]['wind-speed']['value'][j]})
			outDict[locationsList[i]].append({'wind-direction':dataDict['parameters'][i]['direction']['value'][j]})
			outDict[locationsList[i]].append({'time':dataDict['time-layout']['start-valid-time'][j]})
	#Maybe make time stamp a key for other info
	#For structure is:
	"""
	outDict = {
		'point1':{[
			{'loc':{(@latitude, '00.00'), (@longitude, '00.00')}},
			{'wind-speed':'6'}, 
			{'wind-direction':'9'}, 
			{'time':2020-02-06T22:00:00-05:00'}]
		}
		'point2':{same struct as above},
		'pointxx'
	}
	"""
	return outDict


In [111]:
def getDataZipcode(zipcodes, product, begin, end, optional_params=['wspd', 'wdir']):
	params = {
		'zipCodeList': zipcodes,
		'product': product,
		'begin': begin,
		'end': end
	}
	for i in optional_params:
		params[i] = i
	return urlencode(params)

In [209]:
data2 = run_request(getDataZipcode('22152 22150', 'time-series', '2020-05-21T12:00:00', '2020-05-22T17:00:00' ))

http://www.weather.gov/forecasts/xml/sample_products/browser_interface/ndfdXMLclient.php?zipCodeList=22152+22150&product=time-series&begin=2020-05-21T12%3A00%3A00&end=2020-05-22T17%3A00%3A00&wspd=wspd&wdir=wdir


In [229]:
import json
o = xmltodict.parse(data2.content)
d = json.dumps(o)
d = json.loads(d)
# print(d)
print(d['dwml']['data'])

{'location': [{'location-key': 'point1', 'point': {'@latitude': '38.79', '@longitude': '-77.18'}}, {'location-key': 'point2', 'point': {'@latitude': '38.79', '@longitude': '-77.18'}}], 'moreWeatherInformation': [{'@applicable-location': 'point1', '#text': 'https://forecast-v3.weather.gov/point/38.79,-77.18'}, {'@applicable-location': 'point2', '#text': 'https://forecast-v3.weather.gov/point/38.79,-77.18'}], 'time-layout': {'@time-coordinate': 'local', '@summarization': 'none', 'layout-key': 'k-p3h-n41-1', 'start-valid-time': ['2020-05-19T20:00:00-04:00', '2020-05-19T23:00:00-04:00', '2020-05-20T02:00:00-04:00', '2020-05-20T05:00:00-04:00', '2020-05-20T08:00:00-04:00', '2020-05-20T11:00:00-04:00', '2020-05-20T14:00:00-04:00', '2020-05-20T17:00:00-04:00', '2020-05-20T20:00:00-04:00', '2020-05-20T23:00:00-04:00', '2020-05-21T02:00:00-04:00', '2020-05-21T05:00:00-04:00', '2020-05-21T08:00:00-04:00', '2020-05-21T11:00:00-04:00', '2020-05-21T14:00:00-04:00', '2020-05-21T17:00:00-04:00', '202

In [66]:
data = run_request(singlePointDataQuery('39.0000', '-77.0000','time-series', '2020-05-21T12:00:00', '2020-05-22T17:00:00'))

http://www.weather.gov/forecasts/xml/sample_products/browser_interface/ndfdXMLclient.php?lat=39.0000&lon=-77.0000&product=time-series&begin=2020-05-21T12:00:00&end=2020-05-22T17:00:00&Unit=m&wspd=wspd&wdir=wdir


In [67]:
parseXml(data)

OrderedDict([('location', OrderedDict([('location-key', 'point1'), ('point', OrderedDict([('@latitude', '39.00'), ('@longitude', '-77.00')]))])), ('moreWeatherInformation', OrderedDict([('@applicable-location', 'point1'), ('#text', 'https://forecast-v3.weather.gov/point/39.00,-77.00')])), ('time-layout', OrderedDict([('@time-coordinate', 'local'), ('@summarization', 'none'), ('layout-key', 'k-p3h-n7-1'), ('start-valid-time', ['2020-05-21T14:00:00-04:00', '2020-05-21T17:00:00-04:00', '2020-05-21T20:00:00-04:00', '2020-05-22T02:00:00-04:00', '2020-05-22T08:00:00-04:00', '2020-05-22T14:00:00-04:00', '2020-05-22T20:00:00-04:00'])])), ('parameters', OrderedDict([('@applicable-location', 'point1'), ('wind-speed', OrderedDict([('@type', 'sustained'), ('@units', 'meters/second'), ('@time-layout', 'k-p3h-n7-1'), ('name', 'Wind Speed'), ('value', ['4', '5', '4', '3', '1', '1', '1'])])), ('direction', OrderedDict([('@type', 'wind'), ('@units', 'degrees true'), ('@time-layout', 'k-p3h-n7-1'), ('na

TypeError: string indices must be integers

In [208]:
d = xmltodict.parse(data.content)
outDict = {}
print(d['dwml']['data'].keys())
locs = d['dwml']['data']['location']
locKey = locs['location-key']
lat = (locs['point']['@latitude'])
lon = locs['point']['@longitude']
outDict[locKey] = {'latitude':lat, 'longitude':lon}
outDict[locKey]['data'] = []
print(outDict)
print("*************************")
for i in (d['dwml']['data']['time-layout']['start-valid-time']):
    outDict[locKey]['data'].append([{'datetime':i}])
# print(len(d['dwml']['data']['time-layout']['start-valid-time']))
# weatherData = [0]*len(d['dwml']['data']['time-layout']['start-valid-time'])
# print((outDict[locKey]['data'][0]))
# print(weatherData)
params = list((d['dwml']['data']['parameters'].keys()))
for param in range(1,len(params)):
    idx = 0
    for value in d['dwml']['data']['parameters'][params[param]]['value']:
        outDict[locKey]['data'][idx].append({params[param]:value})
        idx+=1
    


odict_keys(['location', 'moreWeatherInformation', 'time-layout', 'parameters'])
{'point1': {'latitude': '39.00', 'longitude': '-77.00', 'data': []}}
*************************
[{'datetime': '2020-05-21T14:00:00-04:00'}, {'wind-speed': '4'}, {'direction': '90'}]


In [147]:
d2 = xmltodict.parse(data2.content)
print(d2['dwml']['data'].keys())
locs = d2['dwml']['data']['location']
print((locs)[0])
print("*************************")

odict_keys(['location', 'moreWeatherInformation', 'time-layout', 'parameters'])
OrderedDict([('location-key', 'point1'), ('point', OrderedDict([('@latitude', '38.79'), ('@longitude', '-77.18')]))])
*************************
